In [2]:
# Add prereq code to set open API key
# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["OPENAI_API_KEY"] = os.environ.get('API_KEY')
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [3]:
# from langchain.llms import OpenAI # this code has been deprecated since recording.
from langchain.chat_models import ChatOpenAI # this is the replacement 
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from random import sample 

First, let's create a loader and load reviews from tv-reviews.csv into memory

In [12]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path='./tv-reviews.csv',
    csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['TV Name', 'Review Title', 'Review Rating', 'Review Text']
})

data = loader.load()

Then, let's initialize our LLM

In [13]:
# TODO: initialize OpenAI object with your API key
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm.openai_api_key = os.environ["OPENAI_API_KEY"] 
llm.openai_api_base = os.environ["OPENAI_API_BASE"]

Now, let's setup our parser and a template  - 

**Note**  that since recording, the code to initialize the model has been updated to 

`llm = ChatOpenAI()`

In [17]:
class ReviewSentiment(BaseModel):
    positives: List[NonNegativeInt] = Field(description="index of a positive TV review, starting from 0")
    negatives: List[NonNegativeInt] = Field(description="index of a negative TV review, starting from 0")
        
parser = PydanticOutputParser(pydantic_object=ReviewSentiment)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"description": "index of a positive TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Positives", "type": "array"}, "negatives": {"description": "index of a negative TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Negatives", "type": "array"}}, "required": ["positives", "negatives"]}
```


Pick 3 sample reviews to classify - LLMs have a limited context window they can work with. In later exercises, we'll see how to deal with that differently

In [15]:
prompt = PromptTemplate(
    template="{question}\n{format_instructions}\nContext: {context}",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)
question = """
    Classify TV reviews provided in the context into positive and negative. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""

reviews_to_classify = sample(data, 3)
context = '\n'.join(review.page_content for review in reviews_to_classify)

query = prompt.format(context = context, question = question)
print(query)


    Classify TV reviews provided in the context into positive and negative. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"description": "index of a positive TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Positives", "type": "array"}, "negatives": {"description": "index of a negative TV review, starting from 

# generate textual prompt from the prompt template
question = """
    Review TVs provided in the context. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""
query = prompt.format(context = context, question = question)

In [10]:
question = """
    Review TVs provided in the context. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""
query = prompt.format(query=question + "\n\n" + str(reviews_to_classify))

Finally, let's send our query to LLM and use the parser we setup to parse an output into a Python object

**NOTE**: Since recording the code to feed the query to the llm has been updated to

`llm.predict(query)`

In [16]:
output = llm.predict(query) #the "predict" has been added
print(output)
result = parser.parse(output)
print(result)
print("Positives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.positives]))
print("Negatives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.negatives]))

/var/folders/2z/b4b21nyd0jxcp1kh766nw8hm0000gp/T/ipykernel_99413/3796528728.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = llm.predict(query) #the "predict" has been added


{
    "positives": [1, 2],
    "negatives": [0]
}
positives=[1, 2] negatives=[0]
Positives:
TV Name: VisionMax Ultra
Review Title: Unmatched Visuals
Review Rating: 10
Review Text: The VisionMax Ultra lives up to its name by delivering unmatched visuals. The 4K resolution and HDR technology bring every scene to life with incredible detail and vibrant colors. It's like having a cinema in my own home. I couldn't be happier with this TV!
TV Name: Imagix Pro
Review Title: Stunning Design
Review Rating: 10
Review Text: The Imagix Pro not only offers exceptional performance but also boasts a stunning design. The ultra-thin bezels and sleek finish make it a focal point of my living room. The TV blends seamlessly with my decor. It's a true work of art!
Negatives:
TV Name: Imagix Pro
Review Title: Impressive Features
Review Rating: 8
Review Text: The Imagix Pro is packed with impressive features that enhance my viewing experience. The smart functionality allows me to easily stream my favorite sh